In [29]:
import  requests
import json
import pandas as pd  
import os 
import mimetypes
import time 


In [2]:
job_dir = "JobDesc"
resume_dir = "Resumes"

In [3]:
urls = {
    "upload_resume" : "https://avermass-cv-ranking.com/uploadfiles/resumes",
    "upload_jobs" : "https://avermass-cv-ranking.com/uploadfiles/jobbdesc",
    "similarity_ranking" : "https://avermass-cv-ranking.com/similarity-rankings"
}

In [31]:
def upload_resume(client_id , resume_path, max_tries = 10):
  payload={}
  mime_type, _ = mimetypes.guess_type(resume_path)

  # If the MIME type cannot be determined, set it to 'application/octet-stream'
  if mime_type is None:
      mime_type = 'application/octet-stream'
  files=[
    (
        'files',
        (
          os.path.basename(resume_path),
          open(resume_path,'rb'),
          mime_type
        )
        ),
    ]
  headers = {}
  url = urls["upload_resume"] + f"?client_id={client_id}"
  for i in range(max_tries):
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    if response.status_code == 200:
      return True
    time.sleep(1)
    print("Retrying ", resume_path)
  print(f"Error uploading {resume_path}", response.status_code, mime_type )
  return False


In [32]:
def upload_jobs(job_path, max_tries = 10):
  payload={}
  mime_type, _ = mimetypes.guess_type(job_path)

  # If the MIME type cannot be determined, set it to 'application/octet-stream'
  if mime_type is None:
      mime_type = 'application/octet-stream'
  files=[
    (
        'files',
        (
          os.path.basename(job_path),
          open(job_path,'rb'),
          mime_type
        )
        ),
    ]
  headers = {}
  url = urls["upload_jobs"] 
  for i in range(max_tries):
    response = requests.request("POST", url, headers=headers, data=payload, files=files)
    if response.status_code == 200:
      return response.json()["client_id"]
    time.sleep(1)
    print("Retrying ", job_path)
  print(f"Error uploading {job_path}", response.status_code, mime_type)
  return False

In [33]:
def get_similarity_ranking(client_id, index = 0 ):
  url = f"https://avermass-cv-ranking.com/similarity-rankings?index={index}&client_id={client_id}"

  payload={'index': '0'}
  files=[

  ]
  headers = {}

  response = requests.request("POST", url, headers=headers, data=payload, files=files)
  if response.status_code == 200:
    return response.json()
  return False

In [34]:
df = []
for job in [os.listdir(job_dir)[0]]:
    job_path = os.path.join(
            os.getcwd(),
            job_dir,
            job
        )
    for resume in os.listdir(resume_dir):

        resume_path = os.path.join(
            os.getcwd(),
            resume_dir,
            resume
        )


        to_append = {
                "jobs" : job_path,
                "Resume File" : resume,
                "Status" : False,
                "Response" : "",
                "client_id" : ""
            }
        client_id = upload_jobs(job_path)
        if client_id:
            to_append["client_id"] = client_id
            if upload_resume(client_id, resume_path, 10):
                results = get_similarity_ranking(client_id=client_id)
                if results:
                    to_append["Status"] =True
                    to_append["Response"] = str(results)
        print(to_append)
        df.append(to_append)


{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '67.pdf', 'Status': True, 'Response': "[{'Name': 'Data/Resumes/08118f0b-b4a9-42fb-8873-3861faea609f/67.docx', 'scores': 51.4656845934, 'rank': 1}]", 'client_id': '08118f0b-b4a9-42fb-8873-3861faea609f'}
{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '59.docx', 'Status': False, 'Response': '', 'client_id': 'f8fd15a1-4deb-4254-a6d0-fa001e31519e'}
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
Retrying  /home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx
{'jobs': '/home/vk001716/ResumeRanking-NLP/Tests/JobDesc/12.docx', 'Resume File': '33.pdf', 'Status': False, 'Response': '', 'client_id': '6c0ad0c0-b63a-4dda-ae8a-9c008fe47bf3'}
{'jobs': '/home/vk001716/ResumeRan

In [37]:
df_ = pd.DataFrame(df)

In [38]:
df_[df_.Status == False].to_csv("failed_results.csv")

In [39]:
df_[df_.Status == False]

,jobs,Resume File,Status,Response,client_id
1,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,59.docx,False,,f8fd15a1-4deb-4254-a6d0-fa001e31519e
2,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,33.pdf,False,,6c0ad0c0-b63a-4dda-ae8a-9c008fe47bf3
5,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,71.pdf,False,,b309ae18-2ee3-4dab-851d-7bea6a3c6c02
11,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,50.pdf,False,,a566e165-90af-447f-94b4-0ca6ebf9c10f
14,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,63.pdf,False,,6b9bca1b-9fab-488e-bd19-5f89bdc64e0c
17,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,31.pdf,False,,964ffa38-53dc-4d7e-bf23-a594ef1d7fe3
19,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,49.pdf,False,,81d56c57-374d-4c6b-a956-9916e1782df3
26,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,53.pdf,False,,48aebd0b-8523-4d20-846a-1dfe1a214595
34,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,5.pdf,False,,f0bea0fd-dfcb-45a5-a6fa-a9af558c90de
42,/home/vk001716/ResumeRanking-NLP/Tests/JobDesc...,25.pdf,False,,2089a3b7-2dde-4360-9a58-b21a7140aa6d
